In [68]:
# -*- coding: utf-8 -*-
import pandas as pd

from bokeh.core.properties import field
from bokeh.io import curdoc
from bokeh.layouts import layout
from bokeh.models import (ColumnDataSource, 
                          HoverTool, 
                          SingleIntervalTicker,
                          Slider, Button, Label, 
                          CategoricalColorMapper)
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from process_data import process_data


#fertility_df, life_expectancy_df, population_df_size, regions_df, years, regions_list = process_data()
x_dim, y_dim, bubble_dim, regions_df, years, regions_list, dims = process_data()
print("Loaded Data.")
list(regions_df[regions_df.index.isin(list(x_dim.index))].Group.unique())
p = pd.Panel({dims[0]: (x_dim / 1000).astype(int), 
              dims[1]: y_dim.astype(int), 
              dims[2]: bubble_dim.astype(int)})

data = {}
region_name = regions_df[regions_df.index.isin(list(x_dim.index))].Group
region_name.name = 'region'
regions_list=list(region_name.unique())
print(regions_list)
for year in years:
    df = pd.concat([p.loc[:, :, year], region_name], axis=1).reset_index()
    data[year] = df.to_dict('series')

print('saved')

Loaded Data.
['East Asia & Pacific', 'Europe & Central Asia', 'North America', 'Latin America & Caribbean ']
saved


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [69]:
data.keys()

dict_keys([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016])

In [70]:
data[1991].get("gdp_pc")

0     28
1     32
2     30
3      9
4     27
5     29
6     21
7     31
8     32
9     22
10    43
11    24
12    30
13    47
14    26
15    36
Name: gdp_pc, dtype: int64

In [71]:
data[1991].get("tech_export")

0      9
1      8
2     15
3      1
4      7
5     18
6     38
7     24
8     17
9      4
10    10
11     7
12    13
13    15
14    24
15    32
Name: tech_export, dtype: int64

In [72]:
dims

['gdp_pc', 'unemployment_rate', 'tech_export']

In [74]:
print(data[1991].get("unemployment_rate"))

0      9
1      5
2     10
3     12
4      6
5      7
6     14
7      2
8      8
9     10
10     6
11    15
12     4
13     1
14     8
15     6
Name: unemployment_rate, dtype: int64


In [75]:
print(x_dim.max().max() / 1000)
print(x_dim.min().min() / 1000)

65.0832586976308
8.99184647717044


In [76]:
print(y_dim.max().max().astype(int))
print(y_dim.min().min().astype(int))

26
0


In [77]:
print("1.")
source = ColumnDataSource(data=data[years[0]])
print("2.")
plot = figure(x_range=(7, 68), y_range=(0.0, 28), title='Gapminder Data', plot_height=300)
plot.xaxis.ticker = SingleIntervalTicker(interval=15000)
plot.xaxis.axis_label = "GDP Per Capita ($USD)"
plot.yaxis.ticker = SingleIntervalTicker(interval=1)
plot.yaxis.axis_label = "Unemployment Rate"
print("3.")
label = Label(x=1.1, y=18, text=str(years[0]), text_font_size='70pt', text_color='#eeeeee')
plot.add_layout(label)
print("4.")
print(regions_list)
print(Spectral6)
color_mapper = CategoricalColorMapper(palette=Spectral6, factors=regions_list)

print("5.")
plot.circle(
    x='gdp_pc',
    y='unemployment_rate',
    size='tech_export',
    source=source,
    fill_color={'field': 'region', 'transform': color_mapper},
    fill_alpha=0.8,
    line_color='#7c7e71',
    line_width=0.5,
    line_alpha=0.5,
    legend=field('region'),
)
plot.add_tools(HoverTool(tooltips="@index", show_arrow=False, point_policy='follow_mouse'))
print("6.")

def animate_update():
    year = slider.value + 1
    if year > years[-1]:
        year = years[0]
    slider.value = year


def slider_update(attrname, old, new):
    year = slider.value
    label.text = str(year)
    source.data = data[year]

slider = Slider(start=years[0], end=years[-1], value=years[0], step=1, title="Year")
slider.on_change('value', slider_update)

callback_id = None

def animate():
    global callback_id
    if button.label == '► Play':
        button.label = '❚❚ Pause'
        callback_id = curdoc().add_periodic_callback(animate_update, 200)
    else:
        button.label = '► Play'
        curdoc().remove_periodic_callback(callback_id)

button = Button(label='► Play', width=60)
button.on_click(animate)

layout = layout([
    [plot],
    [slider, button],
], sizing_mode='scale_width')

#print("7.")
#curdoc().add_root(layout)
#curdoc().title = "Gapminder"

#print("8.")

1.
2.
3.
4.
['East Asia & Pacific', 'Europe & Central Asia', 'North America', 'Latin America & Caribbean ']
['#3288bd', '#99d594', '#e6f598', '#fee08b', '#fc8d59', '#d53e4f']
5.
6.


In [78]:
show(plot)

In [79]:
show(slider)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [80]:
show(button)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [81]:
layout = layout([
    [plot],
    [slider, 
     button],], 
    sizing_mode='stretch_both')

TypeError: 'Column' object is not callable

In [14]:
#l = layout([
#  [bollinger],
#  [sliders, plot],
#  [p1, p2, p3],
#], sizing_mode='stretch_both')

In [82]:
show(layout)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [8]:
handle = show(layout, notebook_handle=True)

# Update the plot title in the earlier cell
#plot.title.text = "New Title"
push_notebook(handle=handle)

ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name: gdp_per_capita, tech_export_rate, unemployment [renderer: GlyphRenderer(id='bdd5a160-8811-46ad-84b2-33e6554cb054', ...)]
You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    http://bokeh.pydata.org/en/latest/docs/user_guide/server.html



In [9]:
curdoc().add_root(layout)
curdoc().title = "Gapminder"

In [12]:
curdoc()